### Завдання 2
Проведіть крос-валідацію для моделі, побудованої на основі датафрейму petrol_consumption з попереднього завдання. Обґрунтуйте обраний метод крос-валідації.


In [57]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn import linear_model, tree, ensemble

#### Імпорт даних, підготовка датасету:

In [58]:
petrol_cons = pd.read_csv('petrol_consumption.csv')
petrol_cons.dropna(axis=0, subset=['Petrol_Consumption'], inplace=True)

y = petrol_cons.Petrol_Consumption          
petrol_cons.drop(['Petrol_Consumption'], axis=1, inplace=True)

numeric_cols = [cname for cname in petrol_cons.columns if petrol_cons[cname].dtype in ['int64', 'float64']]
X = petrol_cons[numeric_cols].copy()

pd.concat([X, y], axis=1).head()

,Petrol_tax,Average_income,Paved_Highways,Population_Driver_licence(%),Petrol_Consumption
0,9.0,3571,1976,0.525,541
1,9.0,4092,1250,0.572,524
2,9.0,3865,1586,0.580,561
3,7.5,4870,2351,0.529,414
4,8.0,4399,431,0.544,410


#### Розділення даних для К-кратної крос-валідації

In [59]:
kf =KFold(n_splits=5, shuffle=True, random_state=5)

print('***************************************************')
print('{}{:^15} {}'.format('Iterations','Training Set', 'Testing Set'))
print('***************************************************')
count = 1
for train_index, test_index in kf.split(X, y):
    print(f'Train {count}   :  ', len(train_index), '         Test:', len(test_index))
    count += 1

***************************************************
Iterations Training Set   Testing Set
***************************************************
Train 1   :   38          Test: 10
Train 2   :   38          Test: 10
Train 3   :   38          Test: 10
Train 4   :   39          Test: 9
Train 5   :   39          Test: 9


#### Древовидна регресія

In [60]:
score = cross_val_score(tree.DecisionTreeRegressor(random_state= 5), X, y, cv=kf, scoring="neg_mean_squared_error")
print(f'Scores for each fold: {score}')
print(f'rmse = {(score.mean()*-1)**0.5}')

Scores for each fold: [-15807.3         -7166.          -9586.2         -6523.77777778
  -4975.88888889]
rmse = 93.87136588616005


In [61]:
max_depth = [1,2,3,4,5,6,7,8,9,10]

rmse_dict ={}
for val in max_depth:
    score = cross_val_score(tree.DecisionTreeRegressor(max_depth= val, random_state= 5), X, y, cv= kf, scoring="neg_mean_squared_error")
    print(f'For max depth: {val}')
    rmse = (score.mean()*-1)**0.5
    print(f'rmse = {rmse}')
    rmse_dict[rmse] = val

print(f'\nМінімальне значення rmse = {min(rmse_dict):.2f} \nГлибина дерева рішень: {rmse_dict[min(rmse_dict)]}')    
    

For max depth: 1
rmse = 99.80850120520277
For max depth: 2
rmse = 91.15402146288433
For max depth: 3
rmse = 97.04707507281668
For max depth: 4
rmse = 90.53644306670793
For max depth: 5
rmse = 90.37699265635575
For max depth: 6
rmse = 93.17066848192682
For max depth: 7
rmse = 93.91929695945358
For max depth: 8
rmse = 94.68341694533656
For max depth: 9
rmse = 87.6828122014546
For max depth: 10
rmse = 93.87136588616005

Мінімальне значення rmse = 87.68 
Глибина дерева рішень: 9


##### Шляхом перебору гіперпараметрів моделі вдалось незначно зменшити квадратичну похибку до 87.68.  Цей показник досягається при максимальній глибині дерева 9.


#### Лінійна регресія

In [62]:
score = cross_val_score(linear_model.LinearRegression(), X, y, cv= kf, scoring="neg_mean_squared_error")
print(f'Scores for each fold: {score}')
print(f'rmse = {(score.mean()*-1)**0.5}')

Scores for each fold: [-3924.50936389 -3305.47069127 -3846.54832979 -2670.99054785
 -9244.86281601]
rmse = 67.81206640240819


#### Регресія на основі випадкового лісу

In [63]:
score = cross_val_score(ensemble.RandomForestRegressor(random_state= 5), X, y, cv= kf, scoring="neg_mean_squared_error")
print(f'Scores for each fold are: {score}')
print(f'rmse = {(score.mean()*-1)**0.5}')

Scores for each fold are: [-10038.32083     -2224.62713     -5854.50417     -2087.64796667
  -9600.35761111]
rmse = 77.20810541358696


In [64]:
estimators = [50, 100, 150, 200, 250, 300, 350, 400]

rmse_dc ={}
for count in estimators:
    score = cross_val_score(ensemble.RandomForestRegressor(n_estimators= count, random_state= 5), X, y, cv= kf, scoring="neg_mean_squared_error")
    print(f'For estimators: {count}')
    rmse = (score.mean()*-1)**0.5
    print(f'rmse = {rmse}')
    rmse_dc[rmse] = count

print(f'\nМінімальне значення rmse = {min(rmse_dc):.2f} \nОцінювач: {rmse_dc[min(rmse_dc)]}')   

For estimators: 50
rmse = 78.56246707486271
For estimators: 100
rmse = 77.20810541358696
For estimators: 150
rmse = 76.07790640301997
For estimators: 200
rmse = 76.23762888094623
For estimators: 250
rmse = 75.76281561892131
For estimators: 300
rmse = 75.18478572568578
For estimators: 350
rmse = 74.82439619447979
For estimators: 400
rmse = 75.01420964325878

Мінімальне значення rmse = 74.82 
Оцінювач: 350


##### Шляхом перебору гіперпараметрів моделі вдалось незначно зменшити квадратичну похибку до 74.82. Цей результат досягається при значенні показника оцінки 350.

#### Моделі випадкових лісів і древовидної регресії вдалося трохи покращити за допомогою тренування. Але все одно, за кінцевими підсумками трьох збудованих моделей найкращі результати показує модель лінійної регресії. Принаймні, стандартна похибка при використанні цієї моделі - мінімальна.
#### Для тренування і оцінювання моделей обрано К-кратну валідацію, оскільки це найпоширеніший метод тренування моделей. Застотсувати стратифіковану крос-валідацію немає можливості, оскільки предиктор не розподілений на певні класи.